In [1]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

/Users/sookim/miniconda3/envs/tcav/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/sookim/miniconda3/envs/tcav/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/sookim/miniconda3/envs/tcav/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/sookim/miniconda3/envs/tcav/lib/python3

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 5621047023196322338]

In [2]:
!git clone https://github.com/pnxenopoulos/cav-keras.git
!cp -R cav-keras/cav cav/

fatal: destination path 'cav-keras' already exists and is not an empty directory.


In [3]:
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras
from keras import Model
from keras.models import Sequential
from keras.layers import InputLayer
from keras.preprocessing.image import ImageDataGenerator

from keras.layers import Dense, Dropout, Activation, Flatten,MaxPool2D, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D

import sys
import os
sys.path.insert(0, os.path.abspath('../..'))

from cav.tcav import *

np.random.seed(1996)

Using TensorFlow backend.


In [4]:
from PIL import Image
# Open the image form working directory
image = Image.open('/Users/sookim/tcav/lung_colon_image_set/colon_image_sets/colon_aca/colonca2122.jpeg')
# summarize some details about the image
print(image.format)
print(image.size)
print(image.mode)

JPEG
(768, 768)
RGB


In [5]:
image_set="/Users/sookim/tcav/lung_colon_image_set/colon_image_sets"
#The subfolder colon_image_sets contains two secondary subfolders: colon_aca subfolder with 5000 images of colon adenocarcinomas and colon_n subfolder with 5000 images of benign colonic tissues.

In [6]:
SIZE_X = SIZE_Y = 128 #384

datagen = tf.keras.preprocessing.image.ImageDataGenerator(validation_split = 0.3,
                                                          )

train_set = datagen.flow_from_directory(image_set,
                                       class_mode = "categorical",
                                       target_size = (SIZE_X,SIZE_Y),
                                       color_mode="rgb",
                                       batch_size = 128, 
                                       shuffle = False,
                                       subset='training',
                                       seed = 42)

validation_set = datagen.flow_from_directory(image_set,
                                       class_mode = "categorical",
                                       target_size = (SIZE_X, SIZE_Y),
                                       color_mode="rgb",
                                       batch_size = 128, 
                                       shuffle = False,
                                       subset='validation',
                                       seed = 42)
test_set = datagen.flow_from_directory(image_set,
                                       class_mode = "binary",
                                       target_size = (SIZE_X, SIZE_Y),
                                       color_mode="rgb",
                                       batch_size = 128, 
                                       shuffle = False,
                                       subset='validation',
                                       seed = 42)

Found 7000 images belonging to 2 classes.
Found 3000 images belonging to 2 classes.
Found 3000 images belonging to 2 classes.


In [7]:
model=Sequential()

model.add(Conv2D(64,(3, 3), padding = 'same', activation='relu', input_shape=(128, 128,3)))
model.add(BatchNormalization())
model.add(Conv2D(128,(3, 3), padding = 'same', activation='relu'))

model.add(MaxPooling2D())
model.add(Dropout(0.1))
model.add(Conv2D(filters = 128, kernel_size = (3, 3), padding = 'same', activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.1))
model.add(Conv2D(filters = 64, kernel_size = (3, 3), padding = 'same', activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.1))
model.add(Conv2D(filters = 64, kernel_size = (3, 3), padding = 'same', activation='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3, 3), padding = 'same', activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(256, activation = 'softmax'))
model.add(Dense(64, activation = 'softmax'))
model.add(Dense(2, activation = 'softmax'))
#model = tf.keras.applications.InceptionV3(
#    include_top=True,
#    weights="imagenet",
#    input_tensor=None,
#    input_shape=None,
#    pooling=None,
#    classes=1000,
#    classifier_activation="softmax",
#)








Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [8]:
es=tf.keras.callbacks.EarlyStopping( monitor='val_accuracy', min_delta=0.001, patience=5, restore_best_weights=True)

In [9]:
model.compile(loss='categorical_crossentropy', optimizer ='adam', metrics=['accuracy'])

In [10]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 128, 128, 64)      1792      
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 128, 64)      256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 128, 128)     73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 128)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 64, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 64, 128)       147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 128)      

In [11]:
model.fit_generator(train_set, validation_data=validation_set, epochs=3, verbose=2,callbacks=[es] )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/3
 - 475s - loss: 0.6944 - acc: 0.5028 - val_loss: 0.6936 - val_acc: 0.5000
Epoch 2/3
 - 434s - loss: 0.6941 - acc: 0.5028 - val_loss: 0.6936 - val_acc: 0.5000
Epoch 3/3
 - 403s - loss: 0.6944 - acc: 0.5028 - val_loss: 0.6935 - val_acc: 0.5000


In [12]:
import cv2
import os

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        img=cv2.resize(img,(128,128))
        if img is not None:
            images.append(img)
    return images


In [13]:
X,y=validation_set.next()

In [14]:
name = ["binucleate_cells","elongated_nuclii","necrotic_debris", "pleomorphism", "signet_ring"]
for n in name:
    concept=load_images_from_folder("/Users/sookim/tcav/lung_colon_image_set/concepts_colon_cancer/cancer/"+n)
    concept=np.array(concept)
    tcav_obj = TCAV()
    tcav_obj.set_model(model)
    print("+++++++++++++++++++++++++++++++++++++\n")
    print(n)
    for i in range(6,  14):
        print("bottleneck layer among", len(model.layers), "is : ", i)
        tcav_obj.split_model(bottleneck = i, conv_layer = True)
        tcav_obj.train_cav(concept)
        tcav_obj.calculate_sensitivity(X, y)
        tcav_obj.print_sensitivity()

+++++++++++++++++++++++++++++++++++++

binucleate_cells
bottleneck layer among 18 is :  6


/Users/sookim/tcav/cav/tcav.py:155: RuntimeWarning: invalid value encountered in long_scalars
  / np.where(self.y_labels == 1)[0].shape[0]


The sensitivity of class 1 is  nan
The sensitivity of class 0 is  0.75
bottleneck layer among 18 is :  7


/Users/sookim/tcav/cav/tcav.py:155: RuntimeWarning: invalid value encountered in long_scalars
  / np.where(self.y_labels == 1)[0].shape[0]


The sensitivity of class 1 is  nan
The sensitivity of class 0 is  0.90625
bottleneck layer among 18 is :  8


...

/Users/sookim/tcav/cav/tcav.py:155: RuntimeWarning: invalid value encountered in long_scalars
  / np.where(self.y_labels == 1)[0].shape[0]


The sensitivity of class 1 is  nan
The sensitivity of class 0 is  1.0
bottleneck layer among 18 is :  12


/Users/sookim/tcav/cav/tcav.py:155: RuntimeWarning: invalid value encountered in long_scalars
  / np.where(self.y_labels == 1)[0].shape[0]


The sensitivity of class 1 is  nan
The sensitivity of class 0 is  0.0
bottleneck layer among 18 is :  13
The sensitivity of class 1 is  nan
The sensitivity of class 0 is  0.0


/Users/sookim/tcav/cav/tcav.py:155: RuntimeWarning: invalid value encountered in long_scalars
  / np.where(self.y_labels == 1)[0].shape[0]
